In [10]:
# Retry: Build Excel workbook and images again if previous attempt was interrupted

import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.styles import numbers
from datetime import datetime
# from ace_tools import display_dataframe_to_user

DATA_DIR = r"C:\Users\maste\Downloads\BI Analyst Capstone Project\New folder"
INPUTS = [
    "sales.csv",
    "product_hierarchy.csv",
    "store_cities.csv",
    "store_names.csv",
    "city_names.csv",
    "product_names.csv",
]

OUTPUT_XLSX = os.path.join(DATA_DIR, "BI_Analyst_Capstone_ML.xlsx")

# ---------------- Helpers ----------------
def load_csv(path):
    try:
        return pd.read_csv(path)
    except Exception:
        try:
            return pd.read_csv(path, encoding="latin-1")
        except Exception as e2:
            raise e2

def detect_date_cols(df: pd.DataFrame):
    cols = []
    for c in df.columns:
        cl = c.lower()
        if any(k in cl for k in ["date", "time", "timestamp", "datetime"]):
            cols.append(c)
    return cols

NUMERIC_STRIP_RE = re.compile(r"[^\d\-\.\,]")

def numericify_object_columns(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.copy()
    for c in df2.columns:
        if pd.api.types.is_object_dtype(df2[c]):
            s = df2[c].astype(str)
            s = s.replace(r"^\s*$", np.nan, regex=True)
            mask = s.notna()
            cleaned = s[mask].str.replace(r"\s+", "", regex=True)
            cleaned = cleaned.apply(lambda x: NUMERIC_STRIP_RE.sub("", x))
            def to_number(x):
                if x.count(",") > 0 and x.count(".") > 0:
                    return x.replace(",", "")
                elif x.count(",") > 0 and x.count(".") == 0:
                    return x.replace(",", ".")
                else:
                    return x
            cleaned = cleaned.apply(to_number)
            converted = pd.to_numeric(cleaned, errors="coerce")
            if (converted.notna().sum() / len(df2)) > 0.3:
                df2.loc[mask, c] = converted
    return df2

def trim_text(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.copy()
    for c in df2.columns:
        if pd.api.types.is_object_dtype(df2[c]):
            df2[c] = df2[c].astype(str).str.strip()
            df2[c] = df2[c].replace(r"^\s*$", np.nan, regex=True)
    return df2

def coerce_dates(df: pd.DataFrame, date_cols):
    df2 = df.copy()
    for c in date_cols:
        try:
            df2[c] = pd.to_datetime(df2[c], errors="coerce", infer_datetime_format=True)
        except Exception:
            pass
    return df2

def missing_counts(df: pd.DataFrame):
    return df.isna().sum().sort_values(ascending=False)

def plot_missing(title, series, out_path):
    plt.figure()
    if series.sum() == 0:
        plt.text(0.5, 0.5, "No missing values", ha="center", va="center")
        plt.title(title)
        plt.axis("off")
    else:
        series = series[series > 0]
        series.plot(kind="bar")
        plt.title(title)
        plt.ylabel("Missing Count")
        plt.tight_layout()
    plt.savefig(out_path, dpi=180, bbox_inches="tight")
    plt.close()

def plot_duplicates(title, count_before, sample_df, out_path):
    plt.figure()
    if count_before == 0 or sample_df is None or sample_df.empty:
        plt.text(0.5, 0.5, "No duplicates", ha="center", va="center")
        plt.title(title)
        plt.axis("off")
    else:
        n = min(10, len(sample_df))
        tbl = plt.table(cellText=sample_df.head(n).astype(str).values,
                        colLabels=sample_df.columns.astype(str),
                        loc="center")
        tbl.auto_set_font_size(False)
        tbl.set_fontsize(6)
        tbl.scale(1, 1.2)
        plt.title(title)
        plt.axis("off")
        plt.tight_layout()
    plt.savefig(out_path, dpi=200, bbox_inches="tight")
    plt.close()

# ---------------- Load all inputs ----------------
datasets = {}
for fname in INPUTS:
    fpath = os.path.join(DATA_DIR, fname)
    if os.path.exists(fpath):
        df = load_csv(fpath)
        datasets[fname] = df
        # display_dataframe_to_user(f"preview_{fname}", df.head(20))
    else:
        datasets[fname] = None

# ---------------- Clean each dataset ----------------
cleaned = {}
summaries = {}
missing_pngs = []
duplicate_pngs = []

for fname, df in datasets.items():
    if df is None:
        continue

    miss_before = missing_counts(df)
    missing_before_png = os.path.join(DATA_DIR, f"{fname.replace('.csv','')}_missing_before.png")
    plot_missing(f"{fname} — Missing Before", miss_before, missing_before_png)
    missing_pngs.append(missing_before_png)

    dup_mask = df.duplicated()
    dup_count_before = int(dup_mask.sum())
    dup_sample = df[dup_mask]
    duplicates_before_png = os.path.join(DATA_DIR, f"{fname.replace('.csv','')}_duplicates_before.png")
    plot_duplicates(f"{fname} — Duplicates Before: {dup_count_before}", dup_count_before, dup_sample, duplicates_before_png)
    duplicate_pngs.append(duplicates_before_png)

    df1 = trim_text(df)
    df2 = numericify_object_columns(df1)
    date_cols = detect_date_cols(df2)
    df3 = coerce_dates(df2, date_cols)
    df4 = df3.dropna(how="any").copy()
    df5 = df4.drop_duplicates().copy()

    cleaned[fname] = df5

    miss_after = missing_counts(df5)
    missing_after_png = os.path.join(DATA_DIR, f"{fname.replace('.csv','')}_missing_after.png")
    plot_missing(f"{fname} — Missing After", miss_after, missing_after_png)
    missing_pngs.append(missing_after_png)

    dup_mask_after = df5.duplicated()
    dup_count_after = int(dup_mask_after.sum())
    dup_sample_after = df5[dup_mask_after] if dup_count_after > 0 else pd.DataFrame()
    duplicates_after_png = os.path.join(DATA_DIR, f"{fname.replace('.csv','')}_duplicates_after.png")
    plot_duplicates(f"{fname} — Duplicates After: {dup_count_after}", dup_count_after, dup_sample_after, duplicates_after_png)
    duplicate_pngs.append(duplicates_after_png)

    summaries[fname] = {
        "rows_before": len(df),
        "rows_after": len(df5),
        "columns": len(df.columns),
        "missing_before_total": int(df.isna().sum().sum()),
        "missing_after_total": int(df5.isna().sum().sum()),
        "duplicates_before": int(df.duplicated().sum()),
        "duplicates_after": int(df5.duplicated().sum())
    }

# ---------------- Write Excel workbook ----------------
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    for fname, df in datasets.items():
        if df is None:
            continue
        base = fname.replace(".csv", "")
        df.head(50000).to_excel(writer, sheet_name=f"{base}_raw", index=False)
        cleaned[fname].to_excel(writer, sheet_name=f"{base}_cleaned", index=False)

    if summaries:
        summ_tbl = pd.DataFrame.from_dict(summaries, orient="index")
        summ_tbl.index.name = "dataset"
        summ_tbl.reset_index(inplace=True)
        summ_tbl.to_excel(writer, sheet_name="cleaning_summary", index=False)

# Create workbook sheet names image
try:
    wb = load_workbook(OUTPUT_XLSX)
    sheet_names = wb.sheetnames
    txt = "Workbook sheets:\n\n" + "\n".join(sheet_names)
except Exception:
    txt = "Workbook not loaded"

sheet_names_img = os.path.join(DATA_DIR, "Workbook_Sheet_Names.png")
plt.figure()
plt.text(0.03, 0.97, txt, va="top")
plt.axis("off")
plt.savefig(sheet_names_img, dpi=180, bbox_inches="tight")
plt.close()

# Show summary in UI
try:
    summary_df = pd.read_excel(OUTPUT_XLSX, sheet_name="cleaning_summary")
    # display_dataframe_to_user("cleaning_summary", summary_df)
except Exception:
    pass

{
    "workbook": OUTPUT_XLSX,
    "sheet_list_image": sheet_names_img,
    "missing_images": missing_pngs,
    "duplicate_images": duplicate_pngs
}


C:\Users\maste\AppData\Local\Temp\ipykernel_6464\2997253402.py:79: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df2[c] = pd.to_datetime(df2[c], errors="coerce", infer_datetime_format=True)


{'workbook': 'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\BI_Analyst_Capstone_ML.xlsx',
 'sheet_list_image': 'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\Workbook_Sheet_Names.png',
 'missing_images': ['C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\sales_missing_before.png',
  'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\sales_missing_after.png',
  'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\product_hierarchy_missing_before.png',
  'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\product_hierarchy_missing_after.png',
  'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\store_cities_missing_before.png',
  'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\store_cities_missing_after.png',
  'C:\\Users\\maste\\Downloads\\BI Analyst Capstone Project\\New folder\\store_names_missing_before.png',
  'C:\\U